In [5]:
# 과제 ver_02 | example 추가 | 히스토리 검색
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=1) # 창의력을 높이기 위해 적정 값 1 적용

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120, 
    return_messages=True,
)

examples = ([
    {"question" : "매트릭스", "answer":"💊💻🕶️"},
    {"question" : "트랜스포머", "answer":"🚗🔧🤖"},
    {"question" : "라이언일병구하기", "answer":"🦁🫡🛡️"},
])

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 질문에 대한 답변을 이모지 3개만을 활용해서 만들어 낼거야"),
        MessagesPlaceholder(variable_name="history"),  # 메모리 히스토리를 옮겨옴
        example_prompt,
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    ) 
    print(result)
    

invoke_chain("스티브잡스")
invoke_chain("빅쇼트")
invoke_chain("중경삼림")	

content='🍏💻📱'
content='💰📉📈'
content='🐯🍃🙇'


In [6]:
print(memory.load_memory_variables({})["history"])

[HumanMessage(content='스티브잡스'), AIMessage(content='🍏💻📱'), HumanMessage(content='빅쇼트'), AIMessage(content='💰📉📈'), HumanMessage(content='중경삼림'), AIMessage(content='🐯🍃🙇')]


In [7]:
# 과제_ver01
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=1) # 창의력을 높이기 위해 적정 값 1 적용

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120, 
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 질문에 대한 답변을 이모지 3개만을 활용해서 만들어 낼거야"),
        MessagesPlaceholder(variable_name="history"),  # 메모리 히스토리를 옮겨옴
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    ) 
    print(result)
    

invoke_chain("트랜스포머")
invoke_chain("매트릭스")
invoke_chain("라이언일병구하기")	
print(memory.load_memory_variables({})["history"])

content='🚗🔥🤖'
content='💊💻🕶️'
content='🦁🙆\u200d♂️🛡️'
[HumanMessage(content='트랜스포머'), AIMessage(content='🚗🔥🤖'), HumanMessage(content='매트릭스'), AIMessage(content='💊💻🕶️'), HumanMessage(content='라이언일병구하기'), AIMessage(content='🦁🙆\u200d♂️🛡️')]
